In [ ]:
from zlib import crc32
import numpy as np

types = ['regression', 'classification']
datasets = {'regression': [{'name': 'Servo Data Set',
                            'url': 'https://archive.ics.uci.edu/ml/datasets/Servo'}, 
                           {'name': 'Forest Fires Data Set',
                            'url': 'https://archive.ics.uci.edu/ml/datasets/Forest+Fires'},
                           {'name': 'Boston Housing Data Set',
                            'url': 'https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html#sklearn.datasets.load_boston'}, 
                           {'name': 'Diabetes Data Set',
                            'url': 'https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes'}],
            'classification': [{'name': 'Spambase Data Set',
                                'url': 'https://archive.ics.uci.edu/ml/datasets/Spambase'}, 
                               {'name': 'Wine Data Set',
                                'url': 'https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine'}, 
                               {'name': 'Breast Cancer Data Set',
                                'url': 'https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer'}, 
                               {'name': 'MNIST',
                                'url': 'http://yann.lecun.com/exdb/mnist/'}]}
methods = {'regression': ['Линейная регрессия', 
                          'Перцептрон', 
                          'Надарая-Ватсона',
                          'SVR'],
           'classification': ['Логистическая регрессия', 
                              'Перцептрон', 
                              'k-ближайших соседей',
                              'Метод потенциальных функций',
                              'Метод Парзеновского окна', 
                              'SVM']}
task = dict()
task['mail'] = input(prompt='Enter your mail: ')
task['id'] = crc32(task['mail'].encode('utf-8'))
np.random.seed(task['id'])
task['type'] = np.random.choice(types)
task['dataset'] = np.random.choice(datasets[task['type']])
task['method'] = np.random.choice(
    methods[task['type']], size=2, replace=False).tolist()


task

Enter your mail: shherbak.ia@phystech.edu


{'dataset': {'name': 'Wine Data Set',
  'url': 'https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine'},
 'id': 3242883651,
 'mail': 'shherbak.ia@phystech.edu',
 'method': ['Метод потенциальных функций', 'k-ближайших соседей'],
 'type': 'classification'}

Щербак Игорь Александрович - 773 группа

Почта -  shherbak.ia@phystech.edu

Датасет - Diabetes Data Set

Методы - Линейная регрессия, Перцептрон

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import sklearn
import torch
import string as s
from sklearn import datasets
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import Perceptron

import warnings
warnings.filterwarnings("ignore")
sklearn.__version__

'0.23.2'

# Внимание!

Для корректной работы дальнейшей программы требуется актуальная версия библиотеки sklearn (0.23 и выше). Данная работы была выполнена в виртуальной среде Google Colabe. Для установки нужной версии библиотеки использовался код из следующей ячейки. К сожалению, я не могу предсказать всевозможные среды, в которых будет проводится проверка данной работы, так что выполнение пункта требований о воспроизводимости кода у преподавателя возможно только при наличии у него актуальной версии библиотки. Надеюсь на понимание, и что все мы идем в ногу со временем)

In [2]:
!pip install scikit-learn -U

     |████████████████████████████████| 6.8MB 4.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
df = load_diabetes(return_X_y=False,as_frame=True)
dataset = pd.DataFrame(df['frame'])
feature_name = pd.DataFrame(df['feature_names'])
data = df['data']
target = df['target']
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.35)


,target
count,155.000000
mean,154.748387
std,78.359693
min,25.000000
25%,88.000000
50%,142.000000
75%,215.500000
max,341.000000




# Анализ данных


Визуально осмотрев входные данные можем сделать вывод, что все они уже приведены к вещественному виду и отнормированны. Они лежат в пределах (0;1) и имеют примерно один порядок $10^{-1}, 10^{-2}$ Биномиальные признаки (пол) так же имеют биномиальные знечения. Так же стоит отметить, что данные не имеют пустых ячеек, что очень удобно, так как не требуется учитывать в работе случаи с пропущенными данными. Однако для оптимизации всё же проведем нормировку данных. Отметим, что нормировка тестовых и тренировочных данных проихводится строго раздельно.



In [3]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train_pd = pd.DataFrame(X_train)
X_train_pd.describe()

,0,1,2,3,4,5,6,7,8,9
count,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02
mean,1.411956e-17,-1.322984e-16,3.674954e-17,-3.868373e-19,-4.642047e-18,-5.802559e-17,3.868373e-19,-7.736746e-18,1.083144e-17,8.510420e-18
std,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00
min,-2.267464e+00,-9.099591e-01,-1.904622e+00,-2.307534e+00,-2.650267e+00,-2.410189e+00,-2.012308e+00,-1.548425e+00,-2.295126e+00,-2.792364e+00
25%,-7.653496e-01,-9.099591e-01,-7.008515e-01,-7.395434e-01,-6.684817e-01,-5.970192e-01,-7.188839e-01,-7.772085e-01,-6.709102e-01,-7.424980e-01
50%,1.205127e-01,-9.099591e-01,-1.620207e-01,-1.456074e-01,-1.147477e-01,-1.073311e-01,-1.645591e-01,-5.992376e-03,-8.734156e-02,5.962343e-02
75%,7.367647e-01,1.098951e+00,6.634221e-01,7.571753e-01,5.555618e-01,5.610270e-01,6.484505e-01,7.652237e-01,6.577264e-01,5.943711e-01
max,2.354426e+00,1.098951e+00,3.437827e+00,2.752800e+00,3.295088e+00,4.233688e+00,3.604849e+00,3.919498e+00,2.999929e+00,2.911611e+00


Но перед началом решения непосредственной задачи, проведем небольшое исследование. Его нет в задании, но проведем его, хуже не будет, и при этом можем получить полезные сведения. Построем простое решающее дерево по всей выборке данных и проверим, какие из признаков вносят какой вклад в общее решение.

In [4]:
from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(random_state=241)
clf.fit(df['data'], df['target'])
importances = clf.feature_importances_
importances = importances*100
print('Вклад признаков в построение прогноза')
for i in range (0,10):
  print(df['feature_names'][i], ' - ' ,"%.2f" %  importances[i], '%')
 
#print(df['feature_names'])
#print(importances)

Вклад признаков в построение прогноза
age  -  3.53 %
sex  -  0.93 %
bmi  -  23.37 %
bp  -  8.65 %
s1  -  8.26 %
s2  -  5.37 %
s3  -  6.74 %
s4  -  1.85 %
s5  -  34.18 %
s6  -  7.14 %


Как мы можем видеть, существенный вклад в вычмсления вносят индекс массы теела и s5Ю, в то время как пол вносит меньше процента в этоговый результат. Следовательно можно попробовать провести анализ выборки без только на основе двух основных признаков илм без учета пола пациента и оценить полученный таким методом результат.

Так же стоит отметить, что не имеет смыла проводить графический анализ даннх и строить графики в двух координатных осях, так как у нас задача регрессии, а не классификации, так что получить наглядного результата, позволющего сделать какие-либо предполодения не получится.

# Линейная регрессия

Начнем с классической линейной регрессии - простая модель, которая в процессе обучения подбирает попарметры весов $w_i$ таких, что $f(x)=w_0+\sum x_iw_i$. Так как линейная регрессия сама по себе не имеет гиперпаметров, которые мы могли бы подобрать, то будем проводить исследования следющим образом: будем искать оптимальное сочетание метода регуляризации и гиперпараметров, ему соответствующих.




In [5]:
param_grid = {
    'alpha': np.arange(0, 4, 0.1), 
    'l1_ratio':  np.arange(0, 1.1, 0.1),
}

gbr_grid_search = GridSearchCV(
                            linear_model.ElasticNet(max_iter=500),
                            param_grid,  
                            cv=5,
                            n_jobs=-1,
                            refit=True,
                            scoring='r2')

gbr_grid_search.fit(X_train, y_train)

print(gbr_grid_search.best_params_)
print(gbr_grid_search.best_score_)
print(gbr_grid_search.best_estimator_)

{'alpha': 1.4000000000000001, 'l1_ratio': 1.0}
0.48481692281901473
ElasticNet(alpha=1.4000000000000001, l1_ratio=1.0, max_iter=500)


Как мы можем видеть, система более тяготеет к L1 регуляризации, так как выбрала максимально возможное значение для li_ratio. Проведем повторное исследование в более мелкой логарифимической сетке в окрестностях полученных точек, что бы определить параметры более точно.

In [6]:
param_grid = {
    'alpha': np.arange(1.3, 1.5, 0.01), 
}

gbr_grid_search = GridSearchCV(
                            linear_model.ElasticNet(max_iter=500,l1_ratio = 1.0),
                            param_grid,  
                            cv=5,
                            n_jobs=-1,
                            refit=True,
                            scoring='r2')

gbr_grid_search.fit(X_train, y_train)

print(gbr_grid_search.best_params_)
print(gbr_grid_search.best_score_)
print(gbr_grid_search.best_estimator_)

best_alpha = gbr_grid_search.best_params_['alpha']
best_score = gbr_grid_search.best_score_

{'alpha': 1.4200000000000002}
0.4848176336027201
ElasticNet(alpha=1.4200000000000002, l1_ratio=1.0, max_iter=500)


---

Посмотрим, как изменится качество алгоритма на оптимальных параметрах, если мы удалим столбец данных, отвечающих за пол или будем использовать только данные о индексе массы тела, и s5. Будем использовать  L1 регулязатор и $\alpha = 0.0613$. 

In [7]:
df = load_diabetes(return_X_y=False,as_frame=True)
kf = KFold(n_splits = 5, shuffle = True, random_state = 100)
data_no_sex = pd.DataFrame(df['data'])
data_only_two = pd.DataFrame(df['data'])
del data_no_sex['sex']                               # создаем первую выборку без пола

L1_reg = linear_model.ElasticNet(alpha = best_alpha, l1_ratio = 1)
quality = cross_val_score(L1_reg, data_no_sex, df['target'], scoring='r2', cv = kf)
print(quality)
av = np.average(quality)
delta =best_score - av
print('Качество системы без данных о поле состваляет ',"%.4f" % av, 'что отличается от качества полной системы на ',"%.4f" % delta)

del data_only_two['age']                            # создаем выборку из двух признаков
del data_only_two['bp']
del data_only_two['s1']
del data_only_two['s2']
del data_only_two['s3']
del data_only_two['s4']
del data_only_two['s6']

quality = cross_val_score(L1_reg, data_only_two, df['target'], scoring='r2', cv = kf)
print(quality)
av = np.average(quality)
delta = best_score - av
print('Качество системы для двух фичейе состваляет ',"%.4f" % av, 'что отличается от качества полной системы на ',"%.4f" % delta)

df = load_diabetes(return_X_y=False,as_frame=True)

[0.25112581 0.2325849  0.21911884 0.19836876 0.20314292]
Качество системы без данных о поле состваляет  0.2209 что отличается от качества полной системы на  0.2639
[0.25112581 0.2325849  0.21911884 0.19836876 0.20314292]
Качество системы для двух фичейе состваляет  0.2209 что отличается от качества полной системы на  0.2639


Как мы можем видеть, предсказание данных без пола и на основе двух признаков пракически не отличаются, следовательно признак пола не играет роли.

---
# Перцептрон
Теперь составим вторую модель - перцептрон. 

In [14]:
class Perceptron(torch.nn.Module):
    def __init__(self, input_dim=10, num_layers=1, 
                 hidden_dim=64, output_dim=347, p=0.0, device='cpu'):
        super(Perceptron, self).__init__()
        
        self.layers = torch.nn.Sequential()
        
        prev_size = input_dim
        for i in range(num_layers):
            self.layers.add_module('layer{}'.format(i), 
                                  torch.nn.Linear(prev_size, hidden_dim))
            self.layers.add_module('relu{}'.format(i), torch.nn.ReLU())
            self.layers.add_module('dropout{}'.format(i), torch.nn.Dropout(p=p))
            prev_size = hidden_dim
        
        self.layers.add_module('classifier', 
                               torch.nn.Linear(prev_size, output_dim))        
        self.to(device)
        
    def forward(self, input):
        return self.layers(input)

In [15]:
def trainer(X_train, y_train, model, loss_function, optimizer, epochs, device='cpu'):
    tensor_x = torch.Tensor(X_train)
    tensor_y = torch.Tensor(y_train.to_numpy())
    dataset = TensorDataset(tensor_x,tensor_y)
    for epoch in tqdm(range(epochs), leave=False):
        generator = torch.utils.data.DataLoader(dataset, batch_size=287, 
                                              shuffle=True)
        for x, y in tqdm(generator, leave=False):
            optimizer.zero_grad()
            x = x.to(device).squeeze().float()
            y = y.to(device)

            output = model(x)
            loss = loss_function(output, y.type(torch.LongTensor))
            loss.backward()
            optimizer.step()

In [16]:
def testing(X_test, y_test, model, device='cpu'):
    tensor_x = torch.Tensor(X_test)
    tensor_y = torch.Tensor(y_test.to_numpy())
    dataset = TensorDataset(tensor_x,tensor_y)
    generator = torch.utils.data.DataLoader(dataset, batch_size=155)

    pred = []
    real = []
    for x, y in tqdm(generator, leave=False):
        x = x.to(device).squeeze().float()
        y = y.to(device)

        pred.extend(torch.argmax(model(x), dim=-1).cpu().numpy().tolist())
        real.extend(y.cpu().numpy().tolist())
    return np.mean(np.array(real) == np.array(pred)), \
           classification_report(real, pred,output_dict= True)

In [17]:
skf = StratifiedKFold(n_splits=4)
accs = []
net = Perceptron(num_layers=1, hidden_dim = 10)
_ = net.train()
trainer(X_train=X_train,
      y_train=y_train,
      model=net, 
      loss_function=torch.nn.CrossEntropyLoss(), 
      optimizer=torch.optim.Adam(net.parameters(), lr=0.001), 
      epochs = 40)
_ = net.eval()
acc, report = testing(X_test, y_test, net)
accs.append(report['accuracy'])



Модель построена и работает, однако дальнейшее исследование и подбор параметров не имеет смысла, так как размерность выходного слоя превышает размерность входного слоя, следовательно - модель математически некорректна.



# Анализ полученных результатов
Как мы можем видеть, качество работы персептрона значительно хуже, чем у линейной регрессии, что полностью совпадает с моими начальными предположениями. Это простой факт обуслаивается тем, что персептрон является методом, исполбующимся в первую очередь в задачах классификации. И хотя технически нашу задачу и можно назвать задачей классивикации (назначение пациенту уровня сахара в крови, выбраному среди предстваленных), все равно у нас будут сотни классов и получить сколько-то верное предсказание для каждого класса на пресептроне невозможно. 


В данном поединке регрессия безоговорочно побеждает, однако схватка была нечестной с самого начала. Однако, почему бы нам ... не изменить правила? Проявим немного инициативы и уровняем шанса.

# Бонусный уровень

Дадим персептрону второй шанс: сведем нашу задачу к задаче классификации, и проведем повторное исследование. Для этого скажем, что пусть нам нужно определить есть ли у человека диабет. Для этого выберем некоторый уровень сахара и скажем, что если у человека в крови его больше, чем в нашем критерии, то он болен, иначе - нет.

In [18]:
y_test_binary = y_test.map(lambda x: 1 if x > 200 else 0)  # создаем новые таргеты
y_train_binary = y_train.map(lambda x: 1 if x > 200 else 0)  # создаем новые таргеты
y_train_binary = pd.Series(y_train_binary).values   
y_test_binary = pd.Series(y_test_binary).values   


Теперь повторим все те же действия, но уже с новыми таргетами.

In [21]:
param_grid = {
    'alpha': np.arange(0, 4, 0.1), 
    'l1_ratio':  np.arange(0, 1.1, 0.1),
}

gbr_grid_search = GridSearchCV(
                            linear_model.ElasticNet(max_iter=500),
                            param_grid,  
                            cv=5,
                            n_jobs=-1,
                            refit=True,
                            scoring='r2')

gbr_grid_search.fit(X_train, y_train_binary)

print(gbr_grid_search.best_params_)
print(gbr_grid_search.best_score_)
print(gbr_grid_search.best_estimator_)




{'alpha': 0.1, 'l1_ratio': 0.0}
0.35077999222828027
ElasticNet(alpha=0.1, l1_ratio=0.0, max_iter=500)
Качество системы упало на  0.1340


Тут метод явно тяготеет к $L_2$ регуляризации

In [22]:
param_grid = {
    'alpha': np.arange(0, 0.2, 0.01), 
}

gbr_grid_search = GridSearchCV(
                            linear_model.ElasticNet(max_iter=500, l1_ratio = 0),
                            param_grid,  
                            cv=5,
                            n_jobs=-1,
                            refit=True,
                            scoring='r2')

gbr_grid_search.fit(X_train, y_train_binary)

print(gbr_grid_search.best_params_)
print(gbr_grid_search.best_score_)
print(gbr_grid_search.best_estimator_)

delta = best_score - gbr_grid_search.best_score_
print('Качество системы упало на ',"%.4f" % delta)

{'alpha': 0.12}
0.3509007761779667
ElasticNet(alpha=0.12, l1_ratio=0, max_iter=500)
Качество системы упало на  0.1339


Как мы видем, качество регрессии для данной задачи резко падает, что опять же логично, так как это метод регрессии, а мы решаем задачу классификации, и пусть мы и получаем довольно точные предсказания 0 и 1, они все равно имеют некоторое отклонение, что вносит свой вклад в функцию ошибки. Это проблема бы решалась, если бы мы просто округляли полученный результат до 0 или 1. Но ведь тогда бы мы просто получили однослойный персептрон)

Теперь очередь персептрона

In [23]:
class Perceptron(torch.nn.Module):
    def __init__(self, input_dim=10, num_layers=1, 
                 hidden_dim=64, output_dim=2, p=0.0, device='cpu'):
        super(Perceptron, self).__init__()
        
        self.layers = torch.nn.Sequential()
        
        prev_size = input_dim
        for i in range(num_layers):
            self.layers.add_module('layer{}'.format(i), 
                                  torch.nn.Linear(prev_size, hidden_dim))
            self.layers.add_module('relu{}'.format(i), torch.nn.ReLU())
            self.layers.add_module('dropout{}'.format(i), torch.nn.Dropout(p=p))
            prev_size = hidden_dim
        
        self.layers.add_module('classifier', 
                               torch.nn.Linear(prev_size, output_dim))        
        self.to(device)
        
    def forward(self, input):
        return self.layers(input)

In [29]:
def trainer(X_train, y_train, model, loss_function, optimizer, epochs, device='cpu'):
    tensor_x = torch.Tensor(X_train)
    tensor_y = torch.Tensor(y_train)
    dataset = TensorDataset(tensor_x,tensor_y)
    for epoch in tqdm(range(epochs), leave=False):
        generator = torch.utils.data.DataLoader(dataset, batch_size=287, 
                                              shuffle=True)
        for x, y in tqdm(generator, leave=False):
            optimizer.zero_grad()
            x = x.to(device).squeeze().float()
            y = y.to(device)

            output = model(x)
            loss = loss_function(output, y.type(torch.LongTensor))
            loss.backward()
            optimizer.step()

In [27]:
def testing(X_test, y_test, model, device='cpu'):
    tensor_x = torch.Tensor(X_test)
    tensor_y = torch.Tensor(y_test)
    dataset = TensorDataset(tensor_x,tensor_y)
    generator = torch.utils.data.DataLoader(dataset, batch_size=155)

    pred = []
    real = []
    for x, y in tqdm(generator, leave=False):
        x = x.to(device).squeeze().float()
        y = y.to(device)

        pred.extend(torch.argmax(model(x), dim=-1).cpu().numpy().tolist())
        real.extend(y.cpu().numpy().tolist())
    return np.mean(np.array(real) == np.array(pred)), \
           classification_report(real, pred,output_dict= True)

In [30]:
skf = StratifiedKFold(n_splits=4)
accs = []
net = Perceptron(num_layers=1, hidden_dim = 10)
_ = net.train()
trainer(X_train=X_train,
      y_train=y_train_binary,
      model=net, 
      loss_function=torch.nn.CrossEntropyLoss(), 
      optimizer=torch.optim.Adam(net.parameters(), lr=0.001), 
      epochs = 40)
_ = net.eval()
acc, report = testing(X_test, y_test_binary, net)
accs.append(report['accuracy'])

print(np.average(acc))

0.7225806451612903




Как мы можем видеть, в данном случае уже персоптрон побеждает, даже если мы не проводили оптимизации гиперпараметров. Но ведь состязание опять было нечестным, так как теперь уже модель линейной регрессии оказалось в проигрошной ситуации из-за задачи классификации.

# Вывод

В условиях поставленной задачи очевидно лучший результат показывает модель линейной регрессии, так как персептрон просто не способен продуктивно работать с задачей регрессии. Что подводит нас к вполне очевидному заключению - при выполнении работы, нужно выбирать рабочую модель в зависимости от поставленных задач и требуемых результатов.

# Проблемы, обнаруженные при выполнении работы:
1. ElasticNet не ставит параметр l1_ratio = 0, хотя по спецификации должен. Для $L_2$ регулязатора пришлось воспользоваться методом Ridge.
2. Так как линейная регрессия выдает вещенственные таргеты, то в ней, даже в случаи задачи классификации мы не можем использовать функцию качества accuracy.
3. В целом можно вообще вынести оценку качества на cross val score к одному большому минусу. Крайне неудобная система метрик, которая накладывает большие ограничения на используемые методы.